<a href="https://colab.research.google.com/github/supsi-dacd-isaac/TeachDecisionMakingUncertainty/blob/main/L01/convex_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed

# Define our functions
def f0(x):
    """Objective function: a "flat" quadratic with minimum at x=2."""
    return (x - 2)**2

def f1(x):
    """Constraint function: a "steep" quadratic.
       The constraint f1(x) <= 0 defines the feasible set."""
    return 3*x**2 - 1

def Lagrangian(x, lam):
    """The Lagrangian function: L(x, lambda) = f0(x) + lambda * f1(x)."""
    return f0(x) + lam * f1(x)

def x_star(lam):
    """Analytically computed minimizer of L(x, lambda).
       Derivation:
           dL/dx = 2(x-2) + 6*lambda*x = 0  =>  x = 2/(1+3*lambda)
    """
    return 2 / (1 + 3*lam)

def d_function(lam):
    """Dual function value d(lambda) = L(x*(lambda), lambda)."""
    xs = x_star(lam)
    return Lagrangian(xs, lam)


In [10]:
#@title Lagrangian Duality: Interactive Notebook Demonstration

def plot_cartesian(lam=0.0):
    # Create a dense array for x-values.
    x_vals = np.linspace(-3, 3, 400)
    f0_vals = f0(x_vals)
    f1_vals = f1(x_vals)
    L_vals  = Lagrangian(x_vals, lam)

    # Compute the current minimizer for the chosen lambda.
    xs = x_star(lam)
    L_min = Lagrangian(xs, lam)

    fig, ax = plt.subplots(1, 2, figsize=(16,5))

    # 1) Add a vertical patch for the region where f1(x) < 0.
    # f1(x) < 0 when 3*x^2 - 1 < 0  =>  |x| < 1/sqrt(3)
    patch_left = -1/np.sqrt(3)
    patch_right = 1/np.sqrt(3)
    ax[0].axvspan(patch_left, patch_right, color='orange', alpha=0.5,
                label=r'Region where $f_1(x)<0$')

    # Plot the original functions and the Lagrangian for the current lambda.
    ax[0].plot(x_vals, f0_vals, label=r'$f_0(x)$', lw=2)
    ax[0].plot(x_vals, f1_vals, label=r'$f_1(x)$', lw=2)
    ax[0].plot(x_vals, L_vals, label=r'$L(x,\lambda)$', lw=2, color='purple', linestyle='--')

    # 2) Plot the locus of the dual function (minima of the Lagrangian)
    # for lambda in the allowed slider range [0, 5].
    lam_range = np.linspace(0, 5, 100)
    x_minima = x_star(lam_range)
    L_minima = Lagrangian(x_minima, lam_range)
    ax[0].plot(x_minima, L_minima, color='green', linestyle='-', lw=2, alpha=0.5,
             label='Dual Function Locus')

    # Mark the current minimizer.
    ax[0].axvline(xs, color='red', linestyle=':', lw=2, label=r'$x^*(\lambda)$')
    ax[0].scatter([xs], [L_min], color='red', s=100, zorder=5)

    ax[0].axhline(0, color='gray', lw=1)
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('Function value')
    ax[0].set_title(f'Functions and Lagrangian (lambda = {lam:.2f})')
    ax[0].legend()
    ax[0].grid(True)
    ax[0].set_xlim(-4, 4)
    ax[0].set_ylim(-2, 25)


    # Compute (f1(x), f0(x)) for a range of x-values.
    x_vals = np.linspace(-3, 3, 400)
    f0_vals = f0(x_vals)
    f1_vals = f1(x_vals)

    # Dual function value d(lambda)
    d_val = d_function(lam)

    # Prepare the supporting line: f0 + lam*f1 = d_val.
    # Solve for f0 = -lam * f1 + d_val.
    # We choose a range for f1 (say, from min to max from our computed values)
    f1_line = np.linspace(f1_vals.min()-0.5, f1_vals.max()+0.5, 100)
    f0_line = -lam * f1_line + d_val

    ax[1].plot(f1_vals, f0_vals, label='Curve: $(f_1(x), f_0(x))$', color='blue')
    ax[1].plot(f1_line, f0_line, label=rf'Supporting Line: $f_0+\lambda f_1={d_val:.2f}$',
             color='red', linestyle='--', lw=2)

    # add a patch for admissible solutions, that is, f1<0
    patch_left = -1e3
    patch_right = 0
    ax[1].axvspan(patch_left, patch_right, color='orange', alpha=0.5,
                label=r'Region where $f_1(x)<0$')

    ax[1].set_xlabel(r'$f_1(x)$')
    ax[1].set_ylabel(r'$f_0(x)$')
    ax[1].set_title(f'Objective Space with Supporting Line (lambda = {lam:.2f})')
    ax[1].legend()
    ax[1].grid(True)
    ax[1].set_xlim(-4, 4)
    ax[1].set_ylim(-20, 20)
    plt.show()
# Create an interactive widget for Plot 1
interact(plot_cartesian, lam=widgets.FloatSlider(min=0, max=5, step=0.1, value=0));



interactive(children=(FloatSlider(value=0.0, description='lam', max=5.0), Output()), _dom_classes=('widget-int…